In [46]:
#Imports and Dependencies

#SQL tools
import sqlalchemy
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

#pandas
import pandas as pd

#to connect to & work with spotify api
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from config import client_id, client_secret
import pprint

#for connecting to postgres/pgadmin
import psycopg2
from config import conn_string ##this one might not work

Pitchfork SQLite tables

In [47]:
#connect to pitchfork database
engine = create_engine("sqlite:///Resources/database.sqlite", echo=False)

#get table names from inspect
inspector = inspect(engine)
inspector.get_table_names()

['artists', 'content', 'genres', 'labels', 'reviews', 'years']

In [48]:
#convert relevant sqlite tables to dataframes
content_df = pd.read_sql('SELECT * FROM content', engine)
genres_df = pd.read_sql('SELECT * FROM genres', engine)
reviews_df = pd.read_sql('SELECT * FROM reviews', engine)

In [49]:
#confirm read_sql worked
reviews_df.head()

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017
2,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017
3,22661,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017
4,22725,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017


In [50]:
#merge relevant pitchfork dfs
merged_df = pd.merge(reviews_df,content_df, how="left", on=["reviewid"])
merged_df = pd.merge(merged_df,genres_df, how="left", on=["reviewid"])
#check merged df
merged_df.head()

,reviewid,title,artist,url,score,best_new_music,author,author_type,pub_date,pub_weekday,pub_day,pub_month,pub_year,content,genre
0,22703,mezzanine,massive attack,http://pitchfork.com/reviews/albums/22703-mezz...,9.3,0,nate patrin,contributor,2017-01-08,6,8,1,2017,"“Trip-hop” eventually became a ’90s punchline,...",electronic
1,22721,prelapsarian,krallice,http://pitchfork.com/reviews/albums/22721-prel...,7.9,0,zoe camp,contributor,2017-01-07,5,7,1,2017,"Eight years, five albums, and two EPs in, the ...",metal
2,22659,all of them naturals,uranium club,http://pitchfork.com/reviews/albums/22659-all-...,7.3,0,david glickman,contributor,2017-01-07,5,7,1,2017,Minneapolis’ Uranium Club seem to revel in bei...,rock
3,22661,first songs,"kleenex, liliput",http://pitchfork.com/reviews/albums/22661-firs...,9.0,1,jenn pelly,associate reviews editor,2017-01-06,4,6,1,2017,Kleenex began with a crash. It transpired one ...,rock
4,22725,new start,taso,http://pitchfork.com/reviews/albums/22725-new-...,8.1,0,kevin lozano,tracks coordinator,2017-01-06,4,6,1,2017,It is impossible to consider a given release b...,electronic


In [51]:
#columns to remove--url, author, author_type, pub_weekday, pub_day, pub_month, pub_year
pitchfork_clean_df = merged_df[['reviewid', 'title', 'artist', 'score', 'best_new_music', 'pub_date', 'content', 'genre']]

len(pitchfork_clean_df)

22710

In [52]:
#remove duplicates
pitchfork_clean_df.drop_duplicates(subset=['reviewid'], keep='first', inplace=True)
print(len(pitchfork_clean_df))
pitchfork_clean_df.head()

18389


C:\Users\ermcg\anaconda3\envs\PythonData\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


,reviewid,title,artist,score,best_new_music,pub_date,content,genre
0,22703,mezzanine,massive attack,9.3,0,2017-01-08,"“Trip-hop” eventually became a ’90s punchline,...",electronic
1,22721,prelapsarian,krallice,7.9,0,2017-01-07,"Eight years, five albums, and two EPs in, the ...",metal
2,22659,all of them naturals,uranium club,7.3,0,2017-01-07,Minneapolis’ Uranium Club seem to revel in bei...,rock
3,22661,first songs,"kleenex, liliput",9.0,1,2017-01-06,Kleenex began with a crash. It transpired one ...,rock
4,22725,new start,taso,8.1,0,2017-01-06,It is impossible to consider a given release b...,electronic


In [53]:
#select out artist column for spotify api call
artist_series = reviews_df['artist']

#check for duplicate artists
print(len(reviews_df['artist']))
print(len(reviews_df['artist'].unique()))

18393
8715


In [54]:
#remove duplicate artists
artist_series_nodupes = artist_series.drop_duplicates(keep='first')

#make it a list
artist_list = artist_series_nodupes.tolist()
len(artist_list)

8715

Spotify API Call

In [44]:
#set spotify client codes as variable, connect spotify to spotipy library
client_credentials_manager = SpotifyClientCredentials(client_id, client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [45]:
##API call takes ~30minute to run

#blank list for storing data
spotify_data = []

for artist in artist_list:
    spotify_dict = {}
   
    try:
        #call api for each artist
        results = sp.search(artist, type='artist')
        #save data in dataframe
        spotify_dict["artist_name"] = results['artists']['items'][0]['name'] 
        spotify_dict["popularity_score"] = results['artists']['items'][0]['popularity']
        spotify_dict["spotify_genres"] = results['artists']['items'][0]['genres']
        spotify_dict["followers"] = results['artists']['items'][0]['followers']['total']

        spotify_data.append(spotify_dict)
    
    except:
        print(f'Error: {artist} is not on spotify.')
       

Error: battle trance is not on spotify.
Error: slbyone is not on spotify.
Error: dm-funk is not on spotify.
Error: sam beam & jesca hoop is not on spotify.
Error: trd, grs och stenar is not on spotify.
Error: black milk & nat turner is not on spotify.
Error: open mike eagle & paul white is not on spotify.
Error: chris forsyth and the solar motel band is not on spotify.
Error: immix ensemble & vessel is not on spotify.
Error: kneebody and daedelus is not on spotify.
Error: dragged into sunlight / gnaw their tongues is not on spotify.
Error: shye ben tzur / jonny greenwood / the rajasthan express is not on spotify.
Error: dm-funk is not on spotify.
Error: joe plummer is not on spotify.
Error: wavves x cloud nothings is not on spotify.
Error: ghostface killah & adrian younge is not on spotify.
Error: todd rundgren/emil nikolaisen/hans-peter lindstrm is not on spotify.
Error: loke rahbek & puce mary is not on spotify.
Error: niia x the range is not on spotify.
Error: livity sound is not on

HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': '', 'limit': 10, 'offset': 0, 'type': 'artist', 'market': None} returned 400 due to No search query


Error:  is not on spotify.


HTTP Error for GET to https://api.spotify.com/v1/search with Params: {'q': '', 'limit': 10, 'offset': 0, 'type': 'artist', 'market': None} returned 400 due to No search query


Error:  is not on spotify.
Error: argentinum astrum is not on spotify.
Error: wooden wand & the world war iv is not on spotify.
Error: livity sound is not on spotify.
Error: dm-funk is not on spotify.
Error: the ex & brass unbound is not on spotify.
Error: oren ambarchi | keiji haino | jim o'rourke is not on spotify.
Error: mark lanegan & duke garwood is not on spotify.
Error: wiz khalifa & curren$y is not on spotify.
Error: ghostface killah & adrian younge is not on spotify.
Error: lafur arnalds is not on spotify.
Error: anas mitchell & jefferson hamer is not on spotify.
Error: adam green & binki shapiro is not on spotify.
Error: pantha du prince & the bell laboratory is not on spotify.
Error: jk flesh / prurient is not on spotify.
Error: boody & le1f is not on spotify.
Error: mad music inc. is not on spotify.
Error: yoko ono / kim gordon / thurston moore is not on spotify.
Error: willits + sakamoto is not on spotify.
Error: rsenkpf is not on spotify.
Error: the electronic anthology p

Error: sunn o))) & boris is not on spotify.
Error: doubled yellow swans is not on spotify.
Error: jeffrey lewis & the jitters is not on spotify.
Error: a cloud mireya is not on spotify.
Error: alias and tarsier is not on spotify.
Error: the gersch is not on spotify.
Error: matt valentine and erika elder is not on spotify.
Error: the congos & friends is not on spotify.
Error: feu thrse is not on spotify.
Error: jarvis cocker & steve mackey is not on spotify.
Error: james f!@.$%^ friedman is not on spotify.
Error: sondre lerche with the faces down quartet is not on spotify.
Error: dell and flgel is not on spotify.
Error: isobel campbell & mark lanegan is not on spotify.
Error: measles mumps rubella is not on spotify.
Error: directing hand is not on spotify.
Error: jenny lewis with the watson twins is not on spotify.
Error: les angles morts is not on spotify.
Error: capillary action is not on spotify.
Error: as mercenrias is not on spotify.
Error: double leopards is not on spotify.
Error:

In [46]:
#turn list of dicts into dataframe to load
spotify_df = pd.DataFrame(spotify_data)
#check dataframe
spotify_df

,artist_name,popularity_score,spotify_genres,followers
0,Massive Attack,66,"[big beat, downtempo, electronica, trip hop]",1791364
1,Krallice,22,"[atmospheric black metal, avant-garde black me...",18358
2,Uranium Club,31,"[art punk, garage pop, garage psych, garage pu...",16552
3,Kleenex,29,"[no wave, post-punk, riot grrrl]",14070
4,Liliput,2,[no wave],1454
...,...,...,...,...
18498,Coldcut,43,"[big beat, electronica, ninja, trip hop, turnt...",41837
18499,Cassius Jay,42,[],4073
18500,Mojave 3,45,"[alternative rock, dream pop, slowcore]",60185
18501,Don Caballero,34,"[emo, instrumental math rock, instrumental roc...",52997


Loading into pgAdmin

In [55]:
#connection to pgAdmin
engine = create_engine(conn_string)

In [56]:
#add pitchfork data
pitchfork_clean_df.to_sql(name='pitchfork_data', con=engine, if_exists='append', index=False)

In [50]:
#add spotify data
spotify_df.to_sql(name='spotify_data', con=engine, if_exists='append', index=False)

In [57]:
#check to make sure files loaded to pgAdmin
pd.read_sql('SELECT * FROM pitchfork_data', engine)

,reviewid,title,artist,score,best_new_music,pub_date,content,genre
0,22703,mezzanine,massive attack,9.3,0,2017-01-08,"“Trip-hop” eventually became a ’90s punchline,...",electronic
1,22721,prelapsarian,krallice,7.9,0,2017-01-07,"Eight years, five albums, and two EPs in, the ...",metal
2,22659,all of them naturals,uranium club,7.3,0,2017-01-07,Minneapolis’ Uranium Club seem to revel in bei...,rock
3,22661,first songs,"kleenex, liliput",9.0,1,2017-01-06,Kleenex began with a crash. It transpired one ...,rock
4,22725,new start,taso,8.1,0,2017-01-06,It is impossible to consider a given release b...,electronic
...,...,...,...,...,...,...,...,...
41094,1535,let us replay!,coldcut,8.9,0,1999-01-26,The marketing guys of yer average modern megac...,jazz
41095,1341,1999,cassius,4.8,0,1999-01-26,"Well, it's been two weeks now, and I guess it'...",electronic
41096,5376,out of tune,mojave 3,6.3,0,1999-01-12,"Out of Tune is a Steve Martin album. Yes, I'l...",rock
41097,2413,"singles breaking up, vol. 1",don caballero,7.2,0,1999-01-12,"Well, kids, I just went back and re-read my re...",rock


In [26]:
#check to make sure files loaded to pgAdmin
pd.read_sql('SELECT * FROM spotify_data', engine)

,artist_name,popularity_score,spotify_genres,followers
0,Massive Attack,66,"{""big beat"",downtempo,electronica,""trip hop""}",1791364
1,Krallice,22,"{""atmospheric black metal"",""avant-garde black ...",18358
2,Uranium Club,31,"{""art punk"",""garage pop"",""garage psych"",""garag...",16552
3,Kleenex,29,"{""no wave"",post-punk,""riot grrrl""}",14070
4,Liliput,2,"{""no wave""}",1454
...,...,...,...,...
18498,Coldcut,43,"{""big beat"",electronica,ninja,""trip hop"",turnt...",41837
18499,Cassius Jay,42,{},4073
18500,Mojave 3,45,"{""alternative rock"",""dream pop"",slowcore}",60185
18501,Don Caballero,34,"{emo,""instrumental math rock"",""instrumental ro...",52997
